In [38]:
# Load gemini
#from context import gemini
from gemini  import data, engine, helpers

# Global Imports
import pandas as pd
import numpy as np

# Build mean reversion strategy
#from talib.abstract import *
import talib

In [52]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.layouts import column
output_notebook()

Loading BokehJS ...

In [46]:
# Higher timeframes (>= daily)
df = data.get_htf_candles("BTC_USD", "Bitfinex", "1-DAY", "2018-02-01 00:00:00", "2019-02-01 00:00:00")
df.head()


ema_20 = df['close'].ewm(span=20,min_periods=0,adjust=False,ignore_na=False).mean()
ema_50 = df['close'].ewm(span=50,min_periods=0,adjust=False,ignore_na=False).mean()

df['EMA_20']= ema_20
df['EMA_50']= ema_50
df.head()

,date,low,high,open,close,volume,EMA_20,EMA_50
0,2018-02-01,8941.00000000,10336.00000000,10267.00000000,9181.00000000,77142.62000000,9181.00000000,9181.00000000
1,2018-02-02,7960.00000000,9213.00000000,9181.00000000,8890.90000000,119927.84000000,9153.37142857,9169.62352941
2,2018-02-03,8180.00000000,9499.00000000,8890.90000000,9225.00000000,50851.27000000,9160.19319728,9171.79515571
3,2018-02-04,7809.00000000,9410.00000000,9225.00000000,8185.20000000,67784.20000000,9067.33670230,9133.10514960
4,2018-02-05,6651.00000000,8380.00000000,8185.20000000,6947.20000000,129428.06000000,8865.41892113,9047.38337903


In [45]:
# Step 1: Find the intersection points 
previous_20 = ema_20.shift(1, axis=0)
previous_50 = ema_50.shift(1, axis=0)


crossing = (((ema_20 <= ema_50) & (previous_20 >= previous_50))
            | ((ema_20 >= ema_50) & (previous_20 <= previous_50)))
df['crossing']=crossing
df.head()

,date,low,high,open,close,volume,EMA_20,EMA_50,crossing
0,2018-02-01,8941.00000000,10336.00000000,10267.00000000,9181.00000000,77142.62000000,9181.00000000,9181.00000000,False
1,2018-02-02,7960.00000000,9213.00000000,9181.00000000,8890.90000000,119927.84000000,9153.37142857,9169.62352941,True
2,2018-02-03,8180.00000000,9499.00000000,8890.90000000,9225.00000000,50851.27000000,9160.19319728,9171.79515571,False
3,2018-02-04,7809.00000000,9410.00000000,9225.00000000,8185.20000000,67784.20000000,9067.33670230,9133.10514960,False
4,2018-02-05,6651.00000000,8380.00000000,8185.20000000,6947.20000000,129428.06000000,8865.41892113,9047.38337903,False


In [41]:
def logic(account, lookback):
    try:
        lookback = helpers.period(lookback)
        today = lookback.loc(0)
        
    #selling #death_cross
        if(today.crossing==True):
            exit_price = today.close
            if(today.EMA_20 > today.EMA_50):
                for position in account.positions:
                    if position.type == 'long':
                        account.close_position(position, 1, exit_price)
                    
        #buying #golden_cross
        if(today.crossing==True):
            
            entry_price = today.open
            risk = 1
            entry_capital = account.buying_power*risk
            if(today.EMA_20 < today.EMA_50):
                if entry_capital > 0:
                    account.enter_position('long', entry_capital, entry_price)
    
    except Exception as e:
        print(e)
        pass

In [42]:
backtest = engine.backtest(df)

In [43]:
output = backtest.start(1000, logic)
output.head()

,benchmark_equity,strategy_equity,benchmark_return,strategy_return
date,,,,
2018-02-01,9181.00000000,1000.00000000,nan,nan
2018-02-02,8890.90000000,1000.00000000,-0.03159787,0.00000000
2018-02-03,9225.00000000,1004.79250626,0.03757775,0.00479251
2018-02-04,8185.20000000,891.53686962,-0.11271545,-0.11271545
2018-02-05,6947.20000000,756.69317068,-0.15124860,-0.15124860


In [44]:
backtest.results()

-------------- Results ----------------

Buy and Hold : -65.57%
Net Profit   : -655.72
Strategy     : -27.23%
Net Profit   : -272.26
Longs        : 5
Sells        : 4
Shorts       : 0
Covers       : 0
--------------------
Total Trades : 9

---------------------------------------


In [72]:
import pyfolio
%matplotlib inline

# silence warnings
import warnings
warnings.filterwarnings('ignore')

In [73]:
pyfolio.create_returns_tear_sheet(output['strategy_return'], benchmark_rets=output['benchmark_return'])

Start date,2018-02-01
End date,2019-02-01
Total months,17
,Backtest
Annual return,-19.7%
Cumulative returns,-27.2%
Annual volatility,55.6%
Sharpe ratio,-0.12
Calmar ratio,-0.37
Stability,0.18
Max drawdown,-52.5%


AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'